In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_x(folder_name, number_images):
    x_input_points = np.zeros((0, 784))
    for i in range(1, number_images + 1): # +1 Since it's exclusive
        img_path = '{}/{}.jpg'.format(folder_name, i)
        x_input_points = np.append(x_input_points, plt.imread(img_path).reshape(1, 784), axis=0)
    # Convert it to binary with threshold 140
    x_input_points[x_input_points < 140] = 0
    x_input_points[x_input_points >= 140] = 1
    return x_input_points

In [ ]:
# Loading in a separate cell to avoid multiple loads.
x_input = get_x('Images', 2400)

In [ ]:
def initialize_center_indices(images, k_number):
    """
    Random initialization of centroids as follows:
        - Start with a random point of the given input points
        - For the rest of K-1 centers remaining keep on:
            + Choose the input point with the maximum distance from the previous
              centroid, making sure the same point is never selected twice.
    """
    number_of_images = images.shape[0]
    centers_indices = np.zeros(k_number).astype(int)
    centers_indices[0] = np.random.randint(0, number_of_images)
    
    for k in range(1, k_number):
        previous_center = x_input[centers_indices[k - 1]]
        # Initialized as the previous center, so that the distance is minimum (0)
        max_so_far = {
            'index': centers_indices[k - 1],
            'value': 0
        }
        for i in range(0, number_of_images):
            difference =  np.dot(previous_center - x_input[i], previous_center - x_input[i])
            if difference > max_so_far['value'] and i not in centers_indices:
                max_so_far = {
                    'index': i,
                    'value': difference
                }
        centers_indices[k] = max_so_far['index']
    return centers_indices

In [ ]:
def get_k_means_clusters(x_input):

    centroids = x_input[initialize_center_indices(x_input, 10)]
    previous_centroids = np.zeros((10, 784))

    # Keep looping till 2 consecutive runs yield the same centroids
    while(not (centroids == previous_centroids).all()):
        previous_centroids = centroids
        images_clusters = np.array([]) # 1d array, size of the count of images (2400)
        for x in x_input:
            differences = np.array([])
            for c in centroids:
                differences = np.append(differences, np.dot(x-c, x-c))

            # Each index represents which cluster an image at said index belongs to
            images_clusters = np.append(images_clusters, np.argmin(differences))

        # Update centroids: get the mean of each cluster(k) 
        centroids = np.array([x_input[images_clusters == k].mean(axis = 0) for k in range(10)])
    
    return centroids

In [ ]:
# Iterate 30 times, the best generated centroids are picked according to the minimum
# sum of squared differences (the closeness of each cluster's members to it's center).

best_centroids = None
smallest_so_far = None
for _ in range(30):
    centroids = get_k_means_clusters(x_input)
    # A 2400 array, each representing the image at index i's cluster number.
    result = np.array([np.argmin([np.dot(x-c, x-c) for c in centroids]) for x in x_input])

    # TODO: Use comprehinsion for extra cool-ness :D
    sums = []
    for k in range(10):
        center_point = centroids[k]
        indices = np.where(result==k)[0] # Indices of images belonging to cluster k
        sums.append(sum(np.dot(x_input[j] - center_point, x_input[j] - center_point) for j in indices))

    if smallest_so_far is None or (sum(sums) < smallest_so_far):
        print('Updated centroids: diff from {} to {}'.format(smallest_so_far, sum(sums)))
        smallest_so_far = sum(sums)
        best_centroids = centroids


In [ ]:
x_clusters = np.array([np.argmin([np.dot(x-c, x-c) for c in best_centroids]) for x in x_input])

# A list of size 10, where the index represents the input digit, and the value is the count of the maximum cluster.
# Eg. For the very first 240 images, the count of the most frequent cluster is the value of the first element.
max_counts = [counts.max() for _, counts in [np.unique(digit, return_counts=True) for digit in np.split(x_clusters, 10)]]

In [ ]:
bars = plt.bar(range(10), height=max_counts, tick_label=[i for i, _ in enumerate(max_counts)])
for i, bar in enumerate(bars):
    y_val = bar.get_height()
    plt.text(bar.get_x() + 0.1, y_val + 1 , max_counts[i]) # This is pure trail and error \_0_/